In [1]:
%%writefile app.py
import streamlit as st
import pprint
import tempfile
import string
import random
import time
import google.generativeai as palm
import os
import pandas as pd
import numpy as np
from langchain.vectorstores import Chroma
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.llms import GooglePalm
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain import HuggingFaceHub, PromptTemplate, LLMChain
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.agents.agent_toolkits import GmailToolkit
from langchain.agents import initialize_agent, AgentType
from io import BytesIO
from langchain.output_parsers import OutputFixingParser
from pyxlsb import open_workbook as open_xlsb
def main():
    st.set_page_config(page_title="HR Assistant", page_icon=":robot:")
    os.environ['GOOGLE_API_KEY'] = 'Enter Your Google API Key'
    palm.configure(api_key=os.environ['GOOGLE_API_KEY'])
    st.session_state.page_dict_HR = {
        "Home":"Job Description",
        "Job Description": "Candidate Resume",
        "Candidate Resume":"Home"
    }
    st.session_state.page_dict_Candidate={
        "Home":"Login Page",
        "Login Page":"Screening Question",
        "Screening Question":"Interview",
        "Interview":"Home"
    }
    if "current_page" not in st.session_state:
        st.session_state.current_page = "Home"
        
    if st.session_state.current_page == "Home":
        Welcome_Page()
        
    elif st.session_state.current_page == "Job Description":
        status=Job_Description_Page()
        if status=="Submitted":
            if st.button("Include Recommendation"):
                status=Job_Description_Page()
        if st.button("Candidate Resume Page"):
            st.session_state.current_page = st.session_state.page_dict_HR[st.session_state.current_page]
            
    elif st.session_state.current_page == "Candidate Resume":
        Candidate_Resume_Page()

       
        if st.button("Home Page"):
            st.session_state.current_page = st.session_state.page_dict_HR[st.session_state.current_page]
            
    elif st.session_state.current_page == "Login Page":
        Login_Page()
        
    elif st.session_state.current_page == "Screening Question":
        Screening_Question()
        if st.button("Start Interview"):
            st.session_state.current_page =  st.session_state.page_dict_Candidate[st.session_state.current_page]
    elif st.session_state.current_page == "Interview":
        Interview()

        if st.button("Home Page"):
            st.session_state.current_page =  st.session_state.page_dict_Candidate[st.session_state.current_page]

def Welcome_Page():
    
    st.markdown(
        """ <style>
        .box{
            position:relative;
            top:40%;
            left:15%;
            font-size:48px;
        }
        .element-container:nth-of-type(3) button{
            width: 200px;
            height: 100px;
            margin: 0 10px;
            font-size: 24px;
            background-color: #65a1f0;
            color: white;
            border: none;
            border-radius: 10px;
            cursor: pointer;
            transition: background-color 0.3s;
            position: relative;
            left: 20%;
        }
        .element-container:nth-of-type(4) button{
            width: 200px;
            height: 100px;
            margin: 0 10px;
            font-size: 24px;
            background-color: #65a1f0;
            color: white;
            border: none;
            border-radius: 10px;
            cursor: pointer;
            transition: background-color 0.3s;
            position: relative;
            left:60%;
            top:-115px;
        }
        
        </style>
        """,
        unsafe_allow_html=True
    )
    
    st.markdown("<p class='box'>Welcome to Interview Portal</p>",unsafe_allow_html=True)
    # Create custom buttons using markdown
    if st.button("HR"):
       st.session_state.current_page = st.session_state.page_dict_HR[st.session_state.current_page]
        
    if st.button("Candidate"):
        st.session_state.current_page = st.session_state.page_dict_Candidate[st.session_state.current_page]




def Job_Description_Page():
    
    st.title("Job Description")
    response_schemas = [
        ResponseSchema(name="Score", description="Score of the Job Description from 1-10 "),
        ResponseSchema(name="Recommendation", description="Suggest specific Improvement in the Job Description")
    ]
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

    def Job_Description(post,query):
      
        model='models/text-bison-001'
        job_Description_llm = GooglePalm(model=model,temperature=0.1)
        template="""Read the job description carefully and identify its strengths and weaknesses.
                Use the following criteria to evaluate the job description on a scale of 1 to 10, where 1 is the lowest and 10 is the highest:
                Clarity: How clear and concise is the job description? Does it avoid jargon, ambiguity and unnecessary details?
                Relevance: How relevant and specific is the job description to the role and the organization? Does it highlight the key skills, responsibilities and qualifications required for the job?
                Attractiveness: How attractive and appealing is the job description to potential candidates? Does it showcase the benefits, perks and opportunities of working for the organization?
                Inclusiveness: How inclusive and diverse is the job description? Does it avoid biased or discriminatory language and promote equal opportunity for all applicants?
                Write a  100 word summary of the score and explain why you gave it. Provide at least 3 relevant recommendation to improve the job description based on your evaluation.
                Example:
                
                Job Description: We are looking for a dynamic and creative Content Writer to join our team. You will be responsible for creating engaging and informative content for our website, blog, social media and newsletters. You will also collaborate with other teams to ensure consistency and quality of our brand voice and tone.
                
                Score: 6/10
                
                Recommendation: The job description is clear and concise, but it lacks relevance and attractiveness. 
                It does not specify what kind of content the writer will create, what topics or industries they will cover, 
                or what goals or metrics they will use to measure their performance. 
                It also does not mention any benefits, perks or opportunities that the organization offers to its employees. 
                A recommendation to improve the job description is to add more details about the role and the organization, 
                such as examples of past projects, clients or achievements, as well as the values, culture and vision of the company. T
                his will help attract more qualified and interested candidates who share the same passion and vision as the organization.
                
                
                 Below Contains the job Description for the job title of {post}\n Job Description: {text}
                 Format for response:
                  Score:
                  Recommendation
                {format_instructions}"""
        prompt_template = PromptTemplate.from_template(template)
        job_messages=prompt_template.format(post=post,text=query,format_instructions=output_parser.get_format_instructions())
        response = job_Description_llm(job_messages)
        new_parser = OutputFixingParser.from_llm(parser=output_parser, llm=GooglePalm())
        response_correct=new_parser.parse(response)
        return response_correct
    submitted=False
    with st.form("myform"):
      st.session_state.post = st.text_input("Enter Job Post:", "")
      st.session_state.query = st.text_area("Enter Job Description:", "")
    
      submitted = st.form_submit_button("Submit")
    Prediction= Job_Description(st.session_state.post,st.session_state.query)
    if(submitted!=False):
        for key, value in Prediction.items():
            st.write(key)
            st.write(value)
        return "Submitted"
        
def Candidate_Resume_Page(): 
    

    st.title("Candidate Resume Scoring")
    if "df_candidates" not in st.session_state:
        st.session_state.df_candidates = pd.DataFrame(columns=["Name","Email ID","Score","Summary"])

    if "count" not in st.session_state:
        st.session_state.count = 0
    if "number_of_resume" not in st.session_state:
        st.session_state.number_of_resume=-1
    hfEmbed = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    def CV_Reader(resume):
      loader = PyPDFLoader(resume)
      pages = loader.load()
      CV_splitter = CharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len)
      docs = CV_splitter.split_documents(pages)
      return docs
    persist_directory = 'docs/chroma/'
    uploaded_files  = st.file_uploader("Upload Candidate's Resume", type=("pdf"), accept_multiple_files=True)
    st.session_state.number_of_resume=len(uploaded_files)
    for uploaded_file in uploaded_files:
        temp_file_path = os.getcwd()
        while uploaded_file is None:
            x = 1
                
        if uploaded_file is not None:
            # Save the uploaded file to a temporary location
            temp_dir = tempfile.TemporaryDirectory()
            temp_file_path = os.path.join(temp_dir.name, uploaded_file.name)
            with open(temp_file_path, "wb") as temp_file:
                temp_file.write(uploaded_file.read())
            docs=CV_Reader(temp_file_path)
            vectordb = Chroma.from_documents(
                documents=docs,
                embedding=hfEmbed,
                persist_directory=persist_directory
            )
           
            
            response_schemas = [
                    ResponseSchema(name="Score", description="Score of the Candidate's CV based alignment of CV with Job Description and Job Post"),
                    ResponseSchema(name="Summary", description="Reason for the Score"),
                    ResponseSchema(name="Name", description="Name of The Candidate"),
                    ResponseSchema(name="Email ID", description="Email ID of the Candidate")]
            output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
            def CV(question):
    
                model='models/text-bison-001'
                fact_llm = GooglePalm(model=model,temperature=0.1)
                
                template = """Act: As an Hr expert
                The CV of the canditate is the context use it to answer the question at the end
                {context}
                Question: {question}
                {format_instructions}
                """
                QA_CHAIN_PROMPT = PromptTemplate.from_template(template,partial_variables={"format_instructions": output_parser.get_format_instructions()})
                qa_chain = RetrievalQA.from_chain_type(
                llm=fact_llm,
                retriever=vectordb.as_retriever(search_kwargs={"k": 1}),
                chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
                )
                result = qa_chain({"query": question})
                return result
            question="""As an HR expert, you are responsible for evaluating candidate CVs and determining their suitability for a specific job opening.
              Your task is to design an efficient scoring system that assesses CVs based on their alignment with the provided job description and job post.
              Create a matching score (ranging from 0 to 100) for each candidate, accurately reflecting their suitability for the position, take into consideration the experience required for the post.
              Consider using both automated scanning methods, such as an Applicant Tracking System (ATS), as well as manual review to ensure comprehensive evaluation.
              Your goal is to facilitate a fair and effective comparison of all candidates, making it easier for the hiring team to identify the most suitable candidates for the job.
              Provide a summary of the Resume along with reason for the score
              Extract the Name of the Canditate
              Extract the Email id of the Canditate
              Format for response:
              Score:
              Summary:
              Name:
              Email ID:
              Below Contains the Job Description along with the Job Post:
             """
            final_query=question+"\n"+"Job Description: "+st.session_state.query+"\n"+"Job Post: "+st.session_state.post
            Prediction= CV(final_query)
            Prediction=output_parser.parse(Prediction["result"])
            columns = list(st.session_state.df_candidates)
            Prediction = {key: value for key, value in Prediction.items() if key in columns}
            st.session_state.df_candidates = pd.concat([st.session_state.df_candidates, pd.DataFrame([Prediction])], ignore_index=True)
            vectordb.delete_collection()
            vectordb.persist()
            st.session_state.count+=1
    if st.session_state.number_of_resume==st.session_state.count:
        
        st.session_state.df_selected = st.session_state.df_candidates[st.session_state.df_candidates['Score'] >= 85]
        def Send_Email_Candidates(email,password):
            model='models/text-bison-001'
            gmail_llm = GooglePalm(model=model,temperature=0.1)
            toolkit = GmailToolkit()
            tools = toolkit.get_tools()
            tool1=[tools[1]]
            template="""Send an Email
            to:goel.chirag34@gmail.com
            subject:Selected for Chat Round
            body: Congratulations you have successfully advanced to the Chat Interview Round for the role {jobpost}. \n\n
                  Here are your credentials for Chat Round.\n\n
                  Username: {EmailID} \n\n
                  Password: {Password} \n\n
            """
            prompt = PromptTemplate.from_template(template)
            agent = initialize_agent(
            tools=tool1,
            llm=gmail_llm,
            agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
            )
            formatted_prompt = prompt.format_prompt(jobpost= st.session_state.post,EmailID=email,Password=password)
            st.write(agent.run(formatted_prompt))
            return
        
        def generate_password(length=8):
            characters = string.ascii_letters + string.digits + string.punctuation
            password = ''.join(random.choice(characters) for _ in range(length))
            return password
            
        def Selection_Interview():
            
            Password=[]
            for index,row in st.session_state.df_selected.iterrows():
                Pass=generate_password()
                Password.append(Pass)
                email=row['Email ID']
                Send_Email_Candidates(email,Pass)
            st.session_state.df_selected['Password']=Password
            return
        
        
       
        
       
        def to_excel(df):
            output = BytesIO()
            writer = pd.ExcelWriter(output, engine='xlsxwriter')
            df.to_excel(writer, index=False, sheet_name='Sheet1')
            workbook = writer.book
            worksheet = writer.sheets['Sheet1']
            format1 = workbook.add_format({'num_format': '0.00'}) 
            worksheet.set_column('A:A', None, format1)  
            writer.close()
            processed_data = output.getvalue()
            return processed_data

        Selection_Interview()
        df_xlsx = to_excel(st.session_state.df_selected)
        st.download_button(label='📥 Download List of Selected Candidates',
                                data=df_xlsx ,
                                file_name= 'List of Selected Candidates.xlsx')
    
        
        
        

def Login_Page():    
    def get_password(username, dataframe):
        try:
            password = dataframe.loc[dataframe['Email ID'] == username, 'Password'].iloc[0]
            return password
        except IndexError:
            return False
    placeholder = st.empty()
    
    # Insert a form in the container
    with placeholder.form("login"):
        st.markdown("#### Enter your credentials")
        email = st.text_input("Email")
        password = st.text_input("Password", type="password")
        submit = st.form_submit_button("Login")
        actual_password=get_password(email,st.session_state.df_selected)
    if submit:
        if password==False:
            st.error("Username Not Found")
        elif password == actual_password:
            placeholder.empty()
            # st.success("Login successful")
            st.session_state.current_page = st.session_state.page_dict_Candidate[st.session_state.current_page]
            if st.session_state.current_page == "Screening Question":
                Screening_Question()
        elif password != actual_password:
            st.error("Login failed")
    else:
        pass

def Screening_Question():   
    st.title("Chat Interview")
    hfEmbed = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    def CV_Reader(resume):
      loader = PyPDFLoader(resume)
      pages = loader.load()
      CV_splitter = CharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len)
      docs = CV_splitter.split_documents(pages)
      return docs
    persist_directory = 'docs/chroma/'
    uploaded_files=None
    uploaded_file=None
    uploaded_files  = st.file_uploader("Upload Candidate's Resume", type=("pdf"), accept_multiple_files=True)
    while uploaded_files is None:
        x = 1
    if uploaded_files is not None:
        for file in uploaded_files:
            uploaded_file=file
    temp_file_path = os.getcwd()        
    if uploaded_file is not None:
        # Save the uploaded file to a temporary location
        temp_dir = tempfile.TemporaryDirectory()
        temp_file_path = os.path.join(temp_dir.name, uploaded_file.name)
        with open(temp_file_path, "wb") as temp_file:
            temp_file.write(uploaded_file.read())
        docs=CV_Reader(temp_file_path)
        st.session_state.vectordb = Chroma.from_documents(
            documents=docs,
            embedding=hfEmbed,
            persist_directory=persist_directory
        )
        df_screening_question=pd.DataFrame(columns=['Question Type','Question','Response'])
        response_schemas = [
        ResponseSchema(name="Essential Questions", description="3 Essential Questions on Job role and Job Description"),
        ResponseSchema(name="Preferred Questions", description="3 Preferred Questions on Job role and Job Description "),
        ResponseSchema(name="Bonus Questions", description="3 Bonus Questions on Job role and Job Description"),
        ResponseSchema(name="CV Specific Questions", description="3 CV specific Questions")]
        output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
        
        def Screening(question):
        # Build prompt
          model='models/text-bison-001'
          fact_llm = GooglePalm(model=model,temperature=0.8)
        
          template = """Act: As an Hr expert
          The CV of the canditate is the context use it to answer the question at the end
        {context}
        Question: {question}
        {format_instructions}
        """
          QA_CHAIN_PROMPT = PromptTemplate.from_template(template,partial_variables={"format_instructions": output_parser.get_format_instructions()})
          #prompt.format(post=post,text=text)
          qa_chain = RetrievalQA.from_chain_type(
            llm=fact_llm,
            retriever=st.session_state.vectordb.as_retriever(search_kwargs={"k": 1}),
            chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
        )
          result = qa_chain({"query": question})
          return result
        question="""As an HR expert, you are tasked with developing screening questions for a job opening in your company.
                You need to consider different levels of importance assigned to the job description and the candidate's CV.
                Your task is to create three sets of screening questions based on the following categories:
                Essential Questions:
                These questions should directly assess the candidate's qualifications for the job role and their alignment with the key requirements of the position. Candidates must answer these questions satisfactorily to move forward in the hiring process.
                Preferred Questions:
                These questions will help you identify candidates who possess additional skills and experiences that would be beneficial for the role but are not strictly required.
                Bonus Questions:
                These are more advanced or specific questions that go beyond the basic requirements of the job description. Candidates who answer these questions well may receive additional consideration during the selection process.
                These questions should be specific to the candidate's CV for the job role.
                In addition to the above categories, please include a "CV Specific Question" in each set.
                This question should be tailored to the individual candidate's CV and experiences, allowing them to showcase their unique strengths and accomplishments relevant to the job role.
                Please create a set of questions for each category, including the CV Specific Question and ensure they align with the job role, job description and the candidates' CV evaluations. Feel free to be creative and thorough in designing questions that will help you identify the best-suited candidates for the position.
                  Format for response:
                  Essential Questions:[3 questions]
                  Preferred Questions:[3 questions]
                  Bonus Questions: [3 questions]
                  CV Specific Questions: [3 questions]
                  Below Contains the Job Description along with the Job role:
                 """
        
       
        final_query=question+"\n"+"Job Description: "+st.session_state.query+"\n"+"Job Post: "+st.session_state.post
        Prediction= Screening(final_query)
        Prediction=output_parser.parse(Prediction["result"])
        for key,value in Prediction.items():
            key=key.strip()
            for v in value:
                v=v.strip()
                new_row = {'Question Type': key, 'Question': v}
                df_screening_question=pd.concat([df_screening_question, pd.DataFrame([new_row])], ignore_index=True)

        st.session_state.Interview_Candidates = df_screening_question
        
            
            
def Interview():
    length_questions_df=len(st.session_state.Interview_Candidates['Question'])            
    def chatbot():
        
        if 'question_index' not in st.session_state:
            st.session_state.question_index = 0
    
        
    # Initialize chat history
        if "messages" not in st.session_state.keys():
            st.session_state.messages = [{"role": "assistant", "content": "Hello! Welcome to the Interview"},                      
                                         {"role": "assistant", "content": "I will be asking you a series of questions"},
                                         {"role": "assistant", "content": "Let's get started!"}
                                        ]
            
        
        
        # Display chat messages from history on app rerun
        for message in st.session_state.messages:
            with st.chat_message(message["role"]):
                st.markdown(message["content"])
        
        # Accept user input
        if st.session_state.question_index <= length_questions_df:
            if prompt := st.chat_input("Enter your Response"):
                # Display user message in chat message container
                with st.chat_message("user"):
                    st.markdown(prompt)
                    
                # Add user message to chat history
                st.session_state.messages.append({"role": "user", "content": prompt})
                if st.session_state.question_index>0:
                    st.session_state.Interview_Candidates.at[st.session_state.question_index-1, 'Response'] = prompt
        
        if st.session_state.messages[-1]["role"] != "assistant":
            if st.session_state.question_index < length_questions_df:
    
                with st.chat_message("assistant"):
                    
                    message_placeholder = st.empty()
                    full_response = ""
                    assistant_response = st.session_state.Interview_Candidates.iloc[st.session_state.question_index]['Question']
                    st.session_state.question_index += 1
                    
                    # Simulate stream of response with milliseconds delay
                    for chunk in assistant_response.split():
                        full_response += chunk + " "
                        time.sleep(0.05)
                        # Add a blinking cursor to simulate typing
                        message_placeholder.markdown(full_response + "▌")
                    message_placeholder.markdown(full_response)
                # Add assistant response to chat history
                st.session_state.messages.append({"role": "assistant", "content": full_response})
            else:
                with st.chat_message('assistant'):
                    st.write('Thank you for answering the questions!')
                    st.write("Your responses have been recorded.")
                    st.write("We appreciate your time.")
                    st.write("Have a great day!")
                    st.session_state.question_index += 1
     
    
    def Evaluation(question):
        
        model='models/text-bison-001'
        fact_llm = GooglePalm(model=model,temperature=0.1)
        
        template = """Act: As an Hr expert
        The CV of the canditate is the context use it to answer the question at the end
        {context}
        Question: {question}
        {format_instructions}
        """
        QA_CHAIN_PROMPT = PromptTemplate.from_template(template,partial_variables={"format_instructions": output_parser.get_format_instructions()})
        #prompt.format(post=post,text=text)
        qa_chain = RetrievalQA.from_chain_type(
        llm=fact_llm,
        retriever=st.session_state.vectordb.as_retriever(search_kwargs={"k": 1}),
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
        )
        result = qa_chain({"query": question})
        return result
    

    def formatted_string():
        formatted_string = ""
    
        for index, row in st.session_state.Interview_Candidates.iterrows():
            formatted_row = "\n".join(f"{column}: {value}" for column, value in row.items())
            formatted_string += formatted_row + "\n"
        
        # Print the formatted string
        return formatted_string
        
    question=""""Imagine you are an HR manager conducting an interview for a position in your company.
                You have received responses from a candidate for some interview questions based on their CV.
                Please evaluate the candidate's responses on a scale of 0 to 100, considering the accuracy of their answers,
                alignment with the job description, genuineness according to their CV, and whether the responses seem to be non-AI generated.
                Provide a brief explanation for your score for each response."
                  Extract the Name of the Canditate
                  Extract the Email id of the Canditate
                  Format for response:
                  Score:
                  Explanation
                  Name:
                  Email ID:
                  Input Format:
                  Type of Question:
                  Question:
                  Response:
                  Below Contains the Job Description along with the Job role:
                     """
    chatbot()
    if st.button("Finish Interview"):
        response_schemas = [
            ResponseSchema(name="Score", description="Score of the Candidate based on his answers alignment to the Job Description and CV"),
            ResponseSchema(name="Explanation", description="Reason for the Score Provided "),
            ResponseSchema(name="Name", description="Name of the Candidate"),
            ResponseSchema(name="Email ID", description="Email Id of the Candidate")]
        output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
        final_query=question+formatted_string()+"Job Description: "+st.session_state.query+"\n"+"Job Post: "+st.session_state.post
        Prediction= Evaluation(final_query)
        Prediction=output_parser.parse(Prediction["result"])
        
        
if __name__=="__main__":
    main()












Writing app.py
